# Test Lidar/GPS positioning performance

Adjusting factors such as: covariance values, downsample rate, Lidar/GPS relative weighting

In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt
from lgchimera.pose_graph import PoseGraph

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
start_idx = 1550

Ground truth trajectory

In [ ]:
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_30_drive_0028_sync', 'oxts', 'data')
gt_data = read_gt(gtpath)
gt_data = gt_data[start_idx:]
lla = gt_data[:,:3]

In [ ]:
from lgchimera.general import lla_to_ecef, ecef2enu

ref_lla = lla[0]
ecef = lla_to_ecef(*lla[0])
gt_ecef = np.zeros((len(lla),3))

for i in range(len(lla)):
    ecef = lla_to_ecef(*lla[i])
    gt_ecef[i] = ecef2enu(ecef[0], ecef[1], ecef[2], ref_lla[0], ref_lla[1], ref_lla[2])

gt_ecef = gt_ecef[:,[1,0,2]]

In [ ]:
from scipy.spatial.transform import Rotation as R

heading = gt_data[0][5] # heading angle
r = R.from_euler('XYZ', [0, 0, heading])
R_heading = r.as_matrix()

In [ ]:
N = len(gt_ecef)
gt_traj = go.Scatter(x=gt_ecef[:,0], y=gt_ecef[:,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

Load LiDAR ICP results

In [ ]:
# Load registration results from file
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', '2011_09_30_drive_0028_sync', 'results', 'p2pl_icp')
lidar_Rs = np.load(os.path.join(data_path, 'lidar_Rs_start_1550_ds_10.npy'))
lidar_ts = np.load(os.path.join(data_path, 'lidar_ts_start_1550_ds_10.npy'))
positions = np.load(os.path.join(data_path, 'positions_start_1550_ds_10.npy'))
lidar_covariances = np.load(os.path.join(data_path, 'covariances_start_1550_ds_10.npy'))
N = len(lidar_Rs)

In [ ]:
lidar_traj = go.Scatter(x=positions[:,0], y=positions[:,1], hovertext=np.arange(N), name='Lidar odometry')
gt_traj = go.Scatter(x=gt_ecef[:N,0], y=gt_ecef[:N,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
gt_traj3d = go.Scatter3d(x=gt_ecef[:,0], y=gt_ecef[:,1], z=gt_ecef[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Ground-truth')
lidar_traj3d = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Lidar odometry')
fig = go.Figure(data=[gt_traj3d, lidar_traj3d])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

## Run Lidar/GPS FGO

In [ ]:
def run_FGO():
    """Run FGO
    
    """
    

Shared parameters

In [ ]:
GPS_SIGMA = 1.5 # [m] (In practice closer to 5)
GPS_INFO = (1/GPS_SIGMA**2) * np.eye(6)

Sliding window

In [ ]:
# Lidar information: fixed vs adaptive
LIDAR_INFO_FIXED = False
LIDAR_INFO = np.eye(6)
LIDAR_INFO[:3,:3] *= 10 

# Downsample rate

# Lidar GPS relative weighting
LIDAR_GPS_WEIGHT = 1.0

In [ ]:
np.random.seed(0)

g = PoseGraph()
N_nodes = 100
window_size = 10

# Add initial node
R_abs = R_heading.copy()
t_abs = gt_ecef[0].copy()
g.add_node(1, (R_abs, t_abs))

# Simulate GPS measurement
gps_pos = gt_ecef[0] + np.random.normal(0, GPS_SIGMA, 3) 
# Add GPS factor and edge
gps_information = np.eye(6)
gps_information[:3,:3] *= 1/GPS_SIGMA**2
g.add_factor(1, (np.eye(3), gps_pos), information=gps_information)

graph_positions = []
gps_measurements = []

# For each new frame
for i in range(1,N_nodes):
    start_time = time.time()

    # Get LiDAR odometry
    R_hat = np.array(lidar_Rs[i-1])
    t_hat = np.array(lidar_ts[i-1])

    # Initialize new node with LiDAR odometry estimate
    R_abs = R_hat @ R_abs
    t_abs += R_abs @ t_hat
    g.add_node(i+1, (R_abs, t_abs))

    # Add LiDAR odometry edge
    if LIDAR_INFO_FIXED:
        lidar_information = LIDAR_INFO
    else:
        lidar_information = np.linalg.inv(lidar_covariances[i-1])
    g.add_edge([i, i+1], (R_hat, t_hat), information=LIDAR_GPS_WEIGHT*lidar_information)

    # Simulate GPS measurement
    gps_pos = gt_ecef[i] + np.random.normal(0, GPS_SIGMA, 3) 
    gps_measurements.append(gps_pos)

    # Add GPS factor and edge
    g.add_factor(i+1, (R_abs, gps_pos), information=GPS_INFO)

    # Trim to window size
    graph_size = max([v.id for v in g.graph._vertices])
    if graph_size > window_size:
        # Save pose before trimming
        graph_positions.append(g.graph._vertices[0].pose.position)
        # Trim
        g.trim_window(window_size)
        # print([v.id for v in g.graph._vertices])
        # print([e.vertex_ids for e in g.graph._edges])

    # Optimize
    # g.graph._link_edges()
    # for e in g.graph._edges:
    #     print(e.calc_chi2_gradient_hessian())
    #print(g.graph._edges[0].calc_error())
    g.optimize(max_iter=5, suppress_output=False)
    #print(g.graph._edges[0].calc_error())

    # Time each iteration
    print(i, "/", N_nodes, ": t=", time.time() - start_time, "s")

graph_positions = np.array(graph_positions)

In [ ]:
#print([e.vertex_ids for e in g.graph._edges])
e = g.graph._edges[0]
print(e.calc_chi2_gradient_hessian())

In [ ]:
chi2 = e.calc_chi2()
err = e.calc_error()
jacobians = e.calc_jacobians()

In [ ]:
e.information

In [ ]:
{v.index: np.dot(np.dot(np.transpose(err), e.information), jacobian) for v, jacobian in zip(e.vertices, jacobians)}

In [ ]:
zip(e.vertices, jacobians)

In [ ]:
{(e.vertices[i].index, e.vertices[j].index): np.dot(np.dot(np.transpose(jacobians[i]), e.information), jacobians[j]) for i in range(len(jacobians)) for j in range(i, len(jacobians))}

In [ ]:
lidar_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N), name='Factor graph')
gt_traj = go.Scatter(x=gt_ecef[:N_nodes,0], y=gt_ecef[:N_nodes,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
Ji = np.zeros((6,6))
Ji[:3,:3] = -np.eye(3)
Jj = np.zeros((6,6))
Jj[:3,:3] = np.eye(3)
J = np.hstack((Ji, Jj))
J.T @ np.eye(6) @ J # H_ij

In [ ]:
Ji = -np.eye(3)
Jj = np.eye(3)
J = np.hstack((Ji, Jj))
J.T @ np.eye(3) @ J

In [ ]:
# RMSE error
rmse_xyz = np.sqrt(np.mean((graph_positions - gt_ecef[:N_nodes-window_size])**2, axis=0))
print("RMSE (xyz): ", rmse_xyz)
print("RMSE (overall): ", rmse_xyz.mean())

Segmented window

In [ ]:
N_nodes = 1000
window_size = 100
num_windows = N_nodes // window_size

# Add initial node
R_abs = R_heading.copy()
t_abs = gt_ecef[0].copy()


graph_positions = []
gps_measurements = []

# For each window
for i in range(num_windows):
    # Form graph over window
    g = PoseGraph()

    # Add initial node
    g.add_node(1, (R_abs, t_abs))

    # Simulate GPS measurement
    gps_pos = gt_ecef[i*window_size] + np.random.normal(0, GPS_SIGMA, 3) 
    # Add GPS factor and edge
    g.add_factor(1, (np.eye(3), gps_pos), information=GPS_INFO)

    for j in range(1,window_size):
        idx = i*window_size + j
        # Get LiDAR odometry
        R_hat = np.array(lidar_Rs[idx-1])
        t_hat = np.array(lidar_ts[idx-1])

        # Initialize new node with LiDAR odometry estimate
        R_abs = R_hat @ R_abs
        t_abs += R_abs @ t_hat
        g.add_node(j+1, (R_abs, t_abs))

        # Add LiDAR odometry edge
        if LIDAR_INFO_FIXED:
            lidar_information = LIDAR_INFO
        else:
            lidar_information = np.linalg.inv(lidar_covariances[idx-1])
        g.add_edge([j, j+1], (R_hat, t_hat), information=LIDAR_GPS_WEIGHT*lidar_information)

        # Simulate GPS measurement
        gps_pos = gt_ecef[idx] + np.random.normal(0, GPS_SIGMA, 3) 
        gps_measurements.append(gps_pos)

        # Add GPS factor and edge 
        g.add_factor(j+1, (R_abs, gps_pos), information=GPS_INFO)

    # Optimize
    g.optimize()

    # Store positions
    graph_positions.append(g.get_positions())

    # Update R_abs and t_abs
    R_abs, t_abs = g.get_poses()[-1]

    # Time each iteration
    print("window", i, "/", num_windows)

graph_positions = np.reshape(graph_positions, (-1,3))
gps_measurements = np.asarray(gps_measurements)

In [ ]:
lidar_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N), name='Factor graph')
gt_traj = go.Scatter(x=gt_ecef[:N_nodes,0], y=gt_ecef[:N_nodes,1], hovertext=np.arange(N), name='Ground-truth')
gps = go.Scatter(x=gps_measurements[:,0], y=gps_measurements[:,1], hovertext=np.arange(N), name='GPS measurements')
fig = go.Figure(data=[gt_traj, lidar_traj, gps])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# RMSE error
rmse_xyz = np.sqrt(np.mean((graph_positions - gt_ecef[:N_nodes])**2, axis=0))
print("RMSE (xyz): ", rmse_xyz)
print("RMSE (overall): ", rmse_xyz.mean())